# Generar N frames sintéticos + "labels"

### Imports

In [20]:
import os
import json
import csv
import pandas as pd
import numpy as np
import mmcv
import keras
import tensorflow as tf
from NDStandardScaler import NDStandardScaler
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import layers

### Paso 1
Procesar los N (14671) frames sintéticos de Blender con un modelo de pose detection ya entrenado 

In [6]:
%run demo/topdown_demo_with_mmdet \
    demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
    https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
    configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py \
    https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth \
    --input inputs/1630-16300.mp4 --save-predictions \
    --output-root outputs/Blender/ 

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14660/14660, 2.0 task/s, elapsed: 7379s, ETA:     0s

C:\Users\crv\miniconda3\envs\openmmlab\lib\site-packages\json_tricks\encoders.py:394: UserWarning: json-tricks: numpy scalar serialization is experimental and may work differently in future versions
  warnings.warn('json-tricks: numpy scalar serialization is experimental and may work differently in future versions')


predictions have been saved at outputs/Blender//results_1630-16300.json


### Paso 2
Entrenar un modelo de aprendizaje supervisado para predecir la posición (x, y) del "heap" del jugador proyectado en la pista a partir de los keypoints de la jugador.

In [60]:
# Obtener los keypoints del fichero json
data = json.load(open('outputs/Blender/results_1630-16300.json'))
# print(data['instance_info'][0]['instances'][0]['keypoints'])
keypoints = []
for i in range(0, len(data['instance_info'])):
    if len(data['instance_info'][i]['instances']) > 0:
        keypoints.append(data['instance_info'][i]['instances'][0]['keypoints'])
    else:
        keypoints.append([])
print(len(keypoints))
# Obtener las posiciones del 'heap' del jugador
ground_truth = list()

with open('outputs/Blender/labels-1630-16300.csv','r') as fp: # open the csv file for reading (will close when "with" block ends)
    for line in fp.readlines(): # go over remaining lines
        col = line.strip().split(";") # get the columns of data
        new_item = [col[1],col[2]] # keep only 2nd and 3th items in a new list, starting count from 0, since this is how lists work in python
        ground_truth.append(new_item) # add the current item to the item list
ground_truth = ground_truth[0:14660]
print(len(ground_truth))

for i in range(0, len(ground_truth)):
    keypoints[i].append(ground_truth[i])
    
# field names 
fields = ['k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'k12', 'k13', 'k14', 'k15', 'k17', 'k18', 'pos']
with open('data.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(keypoints)

14660
14660


In [22]:
# Obtener los frames 
video = mmcv.VideoReader('outputs/Blender/1630-16300.mp4')
video.cvt2frames('outdir')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 14660/14660, 46.8 task/s, elapsed: 313s, ETA:     0s


In [26]:
#dataset = pd.read_csv('data.csv')
# Splitting the Data into Training and Testing
#TargetVariable = ['pos']
#Predictors = ['k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11', 'k12', 'k13', 'k14', 'k15', 'k17', 'k18']

data = json.load(open('outputs/Blender/results_1630-16300.json'))
# print(data['instance_info'][0]['instances'][0]['keypoints'])
zeros = []
X = []
for i in range(0, len(data['instance_info'])):
    if len(data['instance_info'][i]['instances']) > 0:
        X.append(data['instance_info'][i]['instances'][0]['keypoints'])
    else:
        zeros.append(i)
        
zeros.sort(reverse = True)

y = list()
with open('outputs/Blender/labels-1630-16300.csv','r') as fp: # open the csv file for reading (will close when "with" block ends)
    for line in fp.readlines(): # go over remaining lines
        col = line.strip().split(";") # get the columns of data
        new_item = [col[1],col[2]] # keep only 2nd and 3th items in a new list, starting count from 0, since this is how lists work in python
        y.append(new_item) # add the current item to the item list
y = y[0:14660]

for i in zeros:
    y.pop(i)

### Sandardization of data ###
PredictorScaler = NDStandardScaler()
TargetVarScaler = NDStandardScaler()

# Storing the fit object for later reference
PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y)
 
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# create ANN model
model = Sequential()

model.add(tf.keras.layers.Flatten())
 
initializer = tf.keras.initializers.GlorotNormal()
    
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=200, input_dim=17, kernel_initializer=initializer, activation='selu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=200, kernel_initializer=initializer, activation='elu'))
 
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(2, kernel_initializer=initializer))
 
# Compiling the model
model.compile(loss="MeanAbsoluteError", optimizer='rmsprop')
 
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 25, epochs = 20, verbose=1)
Predictions=model.predict(X_test)
print(Predictions[0:10])
MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
print('Accuracy:', 100-MAPE)

(10246, 17, 2)
(10246, 2)
(4392, 17, 2)
(4392, 2)
Epoch 1/20


C:\Users\crv\miniconda3\envs\openmmlab\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


410/410 [==============================] - 1s 998us/step - loss: 0.2093
Epoch 2/20
410/410 [==============================] - 0s 1ms/step - loss: 0.1056
Epoch 3/20
410/410 [==============================] - 0s 997us/step - loss: 0.0939
Epoch 4/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0877
Epoch 5/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0836
Epoch 6/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0797
Epoch 7/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0775
Epoch 8/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0758
Epoch 9/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0744
Epoch 10/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0724
Epoch 11/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0714
Epoch 12/20
410/410 [==============================] - 0s 1ms/step - loss: 0.0703
Epoch 13/20
410/410 [===========

In [13]:
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            
            model.add(tf.keras.layers.Flatten())
            
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
 
            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')
 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
 
            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] )
    return(SearchResultsData)
 
 
######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

138/138 [==============================] - 0s 554us/step
1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 52.013902826252576
138/138 [==============================] - 0s 531us/step
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 48.726907546403076
138/138 [==============================] - 0s 524us/step
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 51.808766890992686
138/138 [==============================] - 0s 534us/step
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 52.52089433952289
138/138 [==============================] - 0s 524us/step
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 51.746367491161735
138/138 [==============================] - 0s 539us/step
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 51.311876586288335
138/138 [==============================] - 0s 531us/step
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 51.41356720939623
138/138 [==============================] - 0s 528us/step
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 5

In [33]:
# Fitting the ANN to the Training set

model.fit(X_train, y_train ,batch_size = 15, epochs = 10, verbose=0)
 
# Generating Predictions on testing data
Predictions=model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)

# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

print(y_test_orig[0:10])
print(Predictions[0:10])

138/138 [==============================] - 0s 655us/step
[[834. 577.]
 [658. 818.]
 [871. 574.]
 [788. 799.]
 [707. 780.]
 [524. 595.]
 [705. 859.]
 [606. 842.]
 [604. 604.]
 [745. 711.]]
[[827.5528  574.988  ]
 [654.24097 816.6083 ]
 [864.3288  564.7999 ]
 [788.4533  800.6785 ]
 [705.0659  776.06757]
 [520.332   590.91095]
 [700.9319  858.5479 ]
 [627.7033  829.4436 ]
 [594.9424  599.2205 ]
 [734.5808  713.46136]]
